In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

In [3]:
from models_torch.torch_original import DeepConvNetTorch
from models.torch_original import DeepConvNetTorch as DeepConvNetNumpy

In [40]:
with open("./Dataset/dummy_data.pkl", 'rb') as f:
    dummy_data = pickle.load(f)

dummy_data = iter(dummy_data)

im_data, gt_boxes, gt_classes, num_obj = next(dummy_data)

In [41]:
im_data, gt_boxes, gt_classes, num_obj = im_data.cuda(), gt_boxes.cuda(), gt_classes.cuda(), num_obj.cuda()
im_data2, gt_boxes2, gt_classes2, num_obj2 = im_data.clone(), gt_boxes.clone(), gt_classes.clone(), num_obj.clone()

In [42]:
import numpy as np
import torch

def compare_outputs(torch_output, numpy_output, function_name, tolerance=1e-5):
    print(f"Comparing outputs for {function_name}:")
    
    if isinstance(torch_output, tuple) and isinstance(numpy_output, tuple):
        for i, (torch_item, numpy_item) in enumerate(zip(torch_output, numpy_output)):
            compare_single_output(torch_item, numpy_item, f"{function_name}[{i}]", tolerance)
    else:
        compare_single_output(torch_output, numpy_output, function_name, tolerance)
    
    print("\n")

def compare_single_output(torch_item, numpy_item, name, tolerance):
    if isinstance(torch_item, torch.Tensor):
        torch_item = torch_item.detach().cpu().numpy()
    if isinstance(numpy_item, torch.Tensor):
        numpy_item = numpy_item.detach().cpu().numpy()
    
    if not isinstance(torch_item, np.ndarray) or not isinstance(numpy_item, np.ndarray):
        print(f"  {name}: Cannot compare. Torch type: {type(torch_item)}, Numpy type: {type(numpy_item)}")
        return
    
    if torch_item.shape != numpy_item.shape:
        print(f"  {name}: Shape mismatch. Torch: {torch_item.shape}, Numpy: {numpy_item.shape}")
        return
    
    abs_diff = np.abs(torch_item - numpy_item)
    max_diff = np.max(abs_diff)
    mean_diff = np.mean(abs_diff)
    
    print(f"  {name}:")
    print(f"    Shape: {torch_item.shape}")
    print(f"    Max absolute difference: {max_diff}")
    print(f"    Mean absolute difference: {mean_diff}")
    
    if max_diff > tolerance:
        print(f"    WARNING: Max difference exceeds tolerance of {tolerance}")
    else:
        print(f"    PASS: All differences within tolerance of {tolerance}")

# Usage in your code:
model_torch = DeepConvNetTorch(input_dims=(3, 416, 416),
                                        num_filters=[16, 32, 64, 128, 256, 512, 1024, 1024],
                                        max_pools=[0, 1, 2, 3, 4],
                                        weight_scale='kaiming',
                                        batchnorm=True,
                                        dtype=torch.float32, device='cuda')
model_numpy = DeepConvNetNumpy(input_dims=(3, 416, 416),
                                        num_filters=[16, 32, 64, 128, 256, 512, 1024, 1024],
                                        max_pools=[0, 1, 2, 3, 4],
                                        weight_scale='kaiming',
                                        batchnorm=True,
                                        dtype=torch.float32, device='cuda')

checkpoint = torch.load('../../yolov2tiny_inference_numpy_version/weights_pytorch/yolov2_epoch_99.pth')
pytorch_model = checkpoint['model']

for param, val in model_torch.params.items():
	for param1, val1 in pytorch_model.items():
		if (param == param1):
			model_torch.params[param] = val1.cuda()

for param, val in model_numpy.params.items():
	for param1, val1 in pytorch_model.items():
		if (param == param1):
			model_numpy.params[param] = val1.cuda()

# Compare Forward
X1 = im_data  # Your input data
X2 = im_data2
out_torch, cache_torch, _ = model_torch.Forward(X1)
out_numpy, cache_numpy, _ = model_numpy.Forward(X2)
compare_outputs((out_torch, cache_torch), (out_numpy, cache_numpy), "Forward")

# Compare loss

loss_torch, loss_grad_torch = model_torch.loss(out_torch, gt_boxes, gt_classes, num_obj)
loss_numpy, loss_grad_numpy = model_numpy.loss(out_numpy, gt_boxes2, gt_classes2, num_obj2)
# compare_outputs((loss_torch, loss_grad_torch), (torch.from_numpy(loss_numpy.data).float(), torch.from_numpy(loss_grad_numpy.data).float()), "loss")
    

Comparing outputs for Forward:
  Forward[0]:
    Shape: (8, 125, 13, 13)
    Max absolute difference: 0.0
    Mean absolute difference: 0.0
    PASS: All differences within tolerance of 1e-05
  Forward[1]: Cannot compare. Torch type: <class 'dict'>, Numpy type: <class 'dict'>


torch.Size([15, 20]) torch.Size([15])
tensor(0.2335, device='cuda:0', grad_fn=<DivBackward0>) tensor(2.1954, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1347, device='cuda:0', grad_fn=<MulBackward0>)
Tensor(0.23352785) Tensor(2.19544194) Tensor(0.13472709)


In [24]:
print(loss_torch, loss_grad_torch.sum())
print()
print(loss_numpy, loss_grad_numpy.sum())

tensor(2.4136, device='cuda:0', grad_fn=<AddBackward0>) tensor(1.0129, device='cuda:0')

Tensor(2.15729963) 1.0129129


In [9]:
print(loss_torch, loss_numpy)

tensor(3.1543, device='cuda:0', grad_fn=<AddBackward0>) Tensor(3.13565084)


In [8]:
dout = ...  # Your upstream gradient
last_dout_torch, grads_torch = model_torch.backward(dout, cache_torch)
last_dout_numpy, grads_numpy = model_numpy.backward(dout, cache_numpy)
compare_outputs((last_dout_torch, grads_torch), (last_dout_numpy, grads_numpy), "backward")

AttributeError: 'ellipsis' object has no attribute 'cuda'